In [4]:
import config
import requests
import praw
import pandas as pd

In [2]:
user_agent = "python:ministryofedCSC_redditscraper:v0.0.1 (by rahm)"

In [3]:
reddit = praw.Reddit(client_id=config.client_id,
                     client_secret=config.client_secret,
                     user_agent=user_agent,
                    )

In [35]:
def extractSubmissionData(rs):
    """
    rs: being a reddit submission type
    return: list of attributes from submission
    """
    return [rs.name, rs.title, rs.subreddit.display_name, rs.author.name,\
            rs.score, rs.upvote_ratio, rs.num_comments, rs.selftext,\
            rs.url, rs.created_utc]

### Get submissions from Reddit

In [33]:
submissionCol = ["id", "title", "subreddit", "author",\
                 "score", "upvote_ratio", "num_comments",\
                 "selftext", "url", "created_utc"]
submissionsData = []


# query parameters
subToSearch = 'ontario'
sort = 'new'
time_filter = 'all'
# limit how many reddit posts you want
limit = 10
# what to search on
query = 'title:education'
lastSubmission = None
while limit > 0:
    if limit > 1000:
        # max limit is 100, so if larger, set a throttled limit
        # and then save the difference
        thrLimit = 1000
        limit -= 1000
    else:
        thrLimit = limit
        limit -= 1000
    
    # additional query params that get passed to an internal class
    # 'praw.models.ListingGenerator'
    queryParams = {}
    if lastSubmission:
        queryParams.update({"after": lastSubmission}) #if lastSubmission != None else pass
    
    
    submissions = reddit.subreddit(subToSearch)\
                    .search(query, sort=sort, time_filter=time_filter,\
                           limit=thrLimit, params=queryParams)
    for sub in submissions:
        print(sub.id)
        attr = extractSubmissionData(sub)
        submissionsData.append(attr)
        # we can use this last submission id as a starting point for our next batch of submissions
        lastSubmission = sub.name

        
submissionsDf = pd.DataFrame(submissionsData, columns=submissionCol)

print(submissionsDf.head())
print(len(submissionsDf))


e9hsir
e8hlj1
e8cfvm
e8c62b
e70pwh
e66hng
e61i2y
e5u67o
e2o4t3
e2k6bj
          id                                              title subreddit  \
0  t3_e9hsir  Ontario staring down a $7-billion bill for edu...   ontario   
1  t3_e8hlj1  Provincial spending could fall far short of de...   ontario   
2  t3_e8cfvm  Del Duca says he would reverse Ford’s educatio...   ontario   
3  t3_e8c62b  11 Lies - Fact-checking the misinformation Edu...   ontario   
4  t3_e70pwh  Shout out to the journalist for calling out Ed...   ontario   

            author  score  upvote_ratio  num_comments selftext  \
0           bitekr      0          0.18            16            
1     GrabbinPills    112          0.97            24            
2  thefrankdomenic     42          0.87            15            
3     Keyboarddesk    708          0.97           234            
4     Keyboarddesk   1192          0.96           394            

                                                 url   created_utc  
0

In [ ]:
commentsCol = ["id", "subreddit", "submission_id", "author",\
              "body", "score", "created_utc", "conversation_id"]
conversationCol = ["id", "top_parent_id", "direct_parent_id",\
                  "child_comment_id"]
commentsData = []
conversationData = []

In [32]:
from collections import deque

def getReplyData(comment, top_level_id):
    """
    comment: redditComment object
    top_level_id: id of the top level comment
    return: list of tuples. tuple[0] is the commentData
            tuple[1] is the conversation data
    """
    commentData = []
    t
def getCommentsFromSubmission(subm_id, top_level_id=None)
    """
    subm_id: submission id
    Return: list of comment objects from given submisison
    """
    # we do this indexing subm_id[2:] because subm_id is
    # the full id of a submission, of the form t2_xxxxx"
    # this reddit.submission method doesnt require the prefix t2_
    subm = reddit.submission(id=subm_id[2:])

    # get rid of 'MoreComments' instances
    subm.comments.replace_more(limit=None)
    # top level comments added to deque datastructure
    
    #comments = deque(subm.comments[:])
    comments = subm.comments


    convo_id = 0

#     while len(comments) > 0:
    for com in comments:
        #com = comments.pop
        comment_data = [com.name, com.subreddit.display_name, subm_id,\
                com.author.name, com.body, com.score,\
                convo_id, com.created_utc]
        
        if top_level_id == None:
            conversation_data = \[convo_id, ]
            
        top_level_id = com.name if top_level_id != None
        
        convo_data = [convo_id, ]
        
    
    

10